In [52]:
import asyncio
import os
import ascon
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from secrets import token_bytes

In [53]:
class AsconCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, nonce, plaintext, associated_data):
        return ascon.ascon_encrypt(self.key, nonce, associated_data, plaintext, variant='Ascon-128')

    def decrypt(self, nonce, ciphertext, associated_data):
        return ascon.ascon_decrypt(self.key, nonce, associated_data, ciphertext, variant='Ascon-128')

In [54]:
async def handle_client(reader, writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

    writer.write(public_key)
    await writer.drain()

    peer_public_key_bytes = await reader.read(32)
    peer_public_key = x25519.X25519PublicKey.from_public_bytes(peer_public_key_bytes)

    shared_key = private_key.exchange(peer_public_key)

    # Derive keys using HKDF
    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=64,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    cipher_key = derived_key_material[:32]

    cipher = AsconCipher(cipher_key)

    while True:
        nonce = await reader.readexactly(16)
        ciphertext = await reader.readuntil(separator=b'|')
        tag = await reader.readexactly(16)
        associated_data = await reader.readuntil(separator=b'||')
        plaintext = cipher.decrypt(nonce, ciphertext, tag, associated_data)

        writer.write(plaintext)
        await writer.drain()

In [55]:
async def main():
    server = await asyncio.start_server(
        handle_client, '127.0.0.1', 8888)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()

In [56]:
async def send_public_key(writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)
    writer.write(public_key)
    await writer.drain()

In [57]:
async def send_message():
    while True:
        try:
            message = input("Enter message to send (or 'quit' to exit): ")
            if message.lower() == 'quit':
                break
            
            reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

            # Send public key
            await send_public_key(writer)

            # Encrypt and send message
            writer.write(message.encode())
            await writer.drain()

            # Receive and decrypt response
            data = await reader.read(1024)  # Increased buffer size for receiving data
            print(f"Received message from server: {data.decode('utf-8', 'ignore')}")  # Decode using UTF-8

            writer.close()
            await writer.wait_closed()
        except KeyboardInterrupt:
            print("Exiting client...")
            break

In [58]:
task = asyncio.create_task(main())

Task exception was never retrieved
future: <Task finished name='Task-25' coro=<main() done, defined at C:\Users\bruna\AppData\Local\Temp\ipykernel_20596\3117860583.py:1> exception=OSError(10048, "error while attempting to bind on address ('127.0.0.1', 8888): normalmente só é permitido uma utilização de cada endereço de socket (protocolo/endereço de rede/porta)")>
Traceback (most recent call last):
  File "C:\Users\bruna\AppData\Local\Temp\ipykernel_20596\3117860583.py", line 2, in main
    server = await asyncio.start_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\streams.py", line 84, in start_server
    return await loop.create_server(factory, host, port, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 153

In [59]:
await asyncio.create_task(send_message())

NameError: name 'ascon_encrypt' is not defined